<a href="https://colab.research.google.com/github/iliasmezzine/SideProjects/blob/master/addCvars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import pandas as pd
import numpy as np

# add_cVars function

# Arguments :

# df (pandas dataframe object): dataframe to be modified
# series (pandas dataframe column) : name of the series on which Var/cVar+/cVar- are calculated
# alphas (tuple with confidence levels)
# days (tuple, past number of historical samples to be included in the rolling window)
# mode (string) : "empirical" returns upper and lower rolling empirical cvar+/- (left mean of x - lower emp. qtile and right mean of x -upper emp. qtile) of series
# mode "theoretical" fits normal distribution to rolling horizon window and returns var(-) / var(+) 

# returns : columns of var+, var-, cvar +, cvar-, for each confidence level, for each horizon, to an existing df, on a given column

def add_cVars(df,series,alphas,days,mode="empirical"):
    
    for day in days:
        for alpha in alphas :
            
            vartxt = "VaR_{}_{}d".format(alpha,day)
            cvartxt = "cVaR_{}_{}d".format(alpha,day)
            vartxt_plus ,vartxt_minus = vartxt + "_(+)", vartxt + "_(-)"
            cvartxt_plus ,cvartxt_minus = cvartxt + "_(+)", cvartxt + "_(-)"
            
            df[vartxt_minus], df[cvartxt_minus] = np.nan, np.nan
            
            if mode == "empirical" :

                #lower and upper empirical quantiles
                #lower and upper empirical cVaRs as defined in article
                
                df[vartxt_plus],  df[cvartxt_plus] = np.nan, np.nan
                
                for i in range(day,len(df)):

                    past = df[series].iloc[i-day:i].sort_values(ascending=True)

                    var_inf = past.iloc[floor(day*alpha)]
                    var_sup = past.iloc[floor(day*(1-alpha))]

                    arr_minus, arr_plus = var_inf - past, past - var_sup

                    df[cvartxt_minus].iloc[i] = arr_minus[arr_minus >= 0].mean()
                    df[cvartxt_plus].iloc[i] = arr_plus[arr_plus >= 0].mean()
                    df[vartxt_plus].iloc[i] = var_sup
                    df[vartxt_minus].iloc[i] = var_inf
                    
            elif mode == "theoretical":

                #theoretical normal quantiles and cVar (they are symmetrical , all the info is in cvar(-) and var(-))
                
                for i in range(day,len(df)):
                    
                    past = df[series].iloc[i-day:i]
                    mu, sig = norm.fit(past)
                    cvar_inf = alpha**-1 * norm.pdf(norm.ppf(alpha))*sig - mu
                    var_inf = norm.ppf(1-alpha)*sig - mu

                    df[cvartxt_minus].iloc[i] = cvar_inf
                    df[vartxt_minus].iloc[i] = var_inf


In [0]:
# Example below on series of normal(0,1)

floor = math.floor
idx = pd.date_range(start='1/1/2018', periods=2500)
rnd = np.random.normal(0,1,len(idx))

df = pd.DataFrame(rnd,index=idx,columns=["Random"])

alphas = [.15]
days = [150]

add_cVars(df,"Random",alphas,days,mode="empirical")

In [28]:
df.dropna().iloc[100:200]

,Random,VaR_0.15_150d_(-),cVaR_0.15_150d_(-),VaR_0.15_150d_(+),cVaR_0.15_150d_(+)
2018-09-08,1.975035,-0.885887,0.422528,1.075518,0.476343
2018-09-09,-1.457497,-0.885887,0.422528,1.078201,0.512769
2018-09-10,0.784854,-0.887528,0.445740,1.078201,0.512769
2018-09-11,0.470263,-0.887528,0.445740,1.078201,0.512769
2018-09-12,-0.359034,-0.887528,0.445740,1.078201,0.512769
...,...,...,...,...,...
2018-12-12,-0.856076,-0.884472,0.445162,1.100661,0.377223
2018-12-13,-1.125847,-0.884472,0.445162,1.100661,0.377223
2018-12-14,0.907263,-0.891793,0.448335,1.100661,0.377223
2018-12-15,-2.436888,-0.891793,0.448335,1.100661,0.377223
